# 使用 Keras 分析 IMDB 电影数据

In [17]:
# Imports
import numpy as np
import keras
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.preprocessing.text import Tokenizer
import matplotlib.pyplot as plt
%matplotlib inline

np.random.seed(42)

## 数据

该数据集预先加载了 Keras，所以一个简单的命令就会帮助我们训练和测试数据。 这里有一个我们想看多少单词的参数。 我们已将它设置为1000，但你可以随时尝试设置为其他数字。

In [18]:
# Loading the data (it's preloaded in Keras)
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=1000)

print(x_train.shape)
print(x_test.shape)

(25000,)
(25000,)


## 检查数据

请注意，数据已经过预处理，其中所有单词都包含数字，评论作为向量与评论中包含的单词一起出现。 例如，如果单词'the'是我们词典中的第一个单词，并且评论包含单词'the'，那么在相应的向量中有 1。

输出结果是 1 和 0 的向量，其中 1 表示正面评论，0 是负面评论。

In [19]:
print(x_train[0])
print(y_train[0])

[1, 14, 22, 16, 43, 530, 973, 2, 2, 65, 458, 2, 66, 2, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 2, 2, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2, 19, 14, 22, 4, 2, 2, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 2, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2, 2, 16, 480, 66, 2, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 2, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 2, 15, 256, 4, 2, 7, 2, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 2, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 2, 88, 12, 16, 283, 5, 16, 2, 113, 103, 32, 15, 16, 2, 19, 178, 32]
1


## 输出的 One-hot 编码

在这里，我们将输入向量转换为 (0,1)-向量。 例如，如果预处理的向量包含数字 14，则在处理的向量中，第 14 个输入将是 1。

In [20]:
# One-hot encoding the output into vector mode, each of length 1000
tokenizer = Tokenizer(num_words=1000)
x_train = tokenizer.sequences_to_matrix(x_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')
print(x_train[0].shape)

(1000,)


In [23]:
x_train.shape, y_train.shape

((25000, 1000), (25000, 2))

同时我们将对输出进行 one-hot 编码。

In [21]:
# One-hot encoding the output
num_classes = 2
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print(y_train.shape)
print(y_test.shape)

(25000, 2)
(25000, 2)


## 模型构建

使用 sequential 在这里构建模型。 请随意尝试不同的层和大小！ 此外，你可以尝试添加 dropout 层以减少过拟合。

In [27]:
# from keras.utils import np_utils

from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout

In [58]:
# TODO: Build the model architecture
model = Sequential()

model.add(Dense(1000, activation='softmax', input_dim=1000))
model.add(Dropout(rate=1-.4))
model.add(Dense(100, activation='tanh'))
# model.add(Dropout(rate=1-.2))
model.add(Dense(10, activation='tanh'))
model.add(Dropout(rate=1-.1))
model.add(Dense(2, activation='sigmoid'))

# TODO: Compile the model using a loss function and an optimizer.
# Compiling the model
model.compile(loss = 'mean_squared_error', optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_45 (Dense)             (None, 1000)              1001000   
_________________________________________________________________
dropout_18 (Dropout)         (None, 1000)              0         
_________________________________________________________________
dense_46 (Dense)             (None, 100)               100100    
_________________________________________________________________
dense_47 (Dense)             (None, 10)                1010      
_________________________________________________________________
dropout_19 (Dropout)         (None, 10)                0         
_________________________________________________________________
dense_48 (Dense)             (None, 2)                 22        
Total params: 1,102,132
Trainable params: 1,102,132
Non-trainable params: 0
_________________________________________________________________


## 训练模型

运行模型。 你可以尝试不同的 batch_size 和 epoch 数量！

In [59]:
%%time
# TODO: Run the model. Feel free to experiment with different batch sizes and number of epochs.
model.fit(x_train, y_train, epochs=20, batch_size=500, verbose=2)

Epoch 1/20
 - 4s - loss: 0.2492 - acc: 0.5229
Epoch 2/20
 - 4s - loss: 0.2211 - acc: 0.6742
Epoch 3/20
 - 3s - loss: 0.1839 - acc: 0.7185
Epoch 4/20
 - 3s - loss: 0.1697 - acc: 0.7322
Epoch 5/20
 - 3s - loss: 0.1630 - acc: 0.7322
Epoch 6/20
 - 4s - loss: 0.1601 - acc: 0.7338
Epoch 7/20
 - 3s - loss: 0.1569 - acc: 0.7339
Epoch 8/20
 - 4s - loss: 0.1525 - acc: 0.7448
Epoch 9/20
 - 3s - loss: 0.1522 - acc: 0.7428
Epoch 10/20
 - 3s - loss: 0.1508 - acc: 0.7447
Epoch 11/20
 - 3s - loss: 0.1475 - acc: 0.7508
Epoch 12/20
 - 3s - loss: 0.1481 - acc: 0.7497
Epoch 13/20
 - 3s - loss: 0.1469 - acc: 0.7468
Epoch 14/20
 - 3s - loss: 0.1464 - acc: 0.7541
Epoch 15/20
 - 3s - loss: 0.1474 - acc: 0.7492
Epoch 16/20
 - 3s - loss: 0.1451 - acc: 0.7542
Epoch 17/20
 - 4s - loss: 0.1447 - acc: 0.7552
Epoch 18/20
 - 4s - loss: 0.1448 - acc: 0.7528
Epoch 19/20
 - 3s - loss: 0.1444 - acc: 0.7491
Epoch 20/20
 - 4s - loss: 0.1420 - acc: 0.7606
Wall time: 1min 10s


## 评估模型

你可以在测试集上评估模型，这将为你提供模型的准确性。你得出的结果可以大于 85% 吗？

In [60]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Accuracy: ", score[1])

Accuracy:  0.85888


```
0.85804, 
```

__END__